# CAMBIO
Explore creating phenopackts from CAMBIO data (prototype, not for final analysis!)

In [1]:
from cambio import Cambio, Cambio2Phenopacket
from pathlib import Path

ImportError: cannot import name 'Cambio2Phenopacket' from 'cambio' (/Users/robin/GIT/cambio-analysis/src/cambio/__init__.py)

In [ ]:
input_file = "../../../Desktop/HPOstuff/2025_07_24_CAMBio_Tabelle.txt"
# "../../../Documents/manuscripts/AMD/2025_07_24_CAMBio_Tabelle_für_Hannes_Urbat.txt"
input_path = Path(input_file).resolve()
cambio = Cambio(cambio_path=input_path)

In [ ]:
cambio2ppkt = Cambio2Phenopacket(df=cambio.get_df())

In [ ]:
ppkt_d = cambio2ppkt.get_ppkt_d()
print(f"Got {len(ppkt_d)} phenopackets")

In [ ]:
from google.protobuf.json_format import MessageToJson
import phenopackets as PPKt

def output_ppkt(ppkt: PPKt.Phenopacket):
    json_string = MessageToJson(ppkt)
    print(json_string)


In [ ]:
cambio2ppkt.add_demographic_data()
cambio2ppkt.add_hpo_columns()

In [ ]:
for k, v in ppkt_d.items():
    #print(f"k={k} v={v}")
    output_ppkt(v)